# Import Libraries

In [ ]:
#DataFrames manipulation
import pandas as pd

#Date manipulation
import datetime

#Data download from Yahoo Finance
from yahooquery import Ticker

#Libraries for the Plotting
import holoviews as hv
from holoviews import opts, dim
from bokeh.models import HoverTool

#Librarie to save the plots to html object
import panel as pn

hv.extension('bokeh')

# Download Data from Yahoo Finance

In [ ]:
#Download the Data from Yahoo Finance
tickers = Ticker('^VIX')
df = tickers.history(start = '1970-01-01')

#Limit the Data to the Close and rearrange columns
df = df.reset_index()
df = df[['date', 'close']]
df = df.rename(columns = {'date': 'Date', 'close': 'VIX'})

# Convert the 'Date' column to datetime format
df["Date"] = pd.to_datetime(df["Date"], utc = 'true')

# Manipulate the Data for the Calendar Year VIX Evolution Graph

In [ ]:
#Create a New Object
pivot_df = df

# Extract year and day components
pivot_df["Year"] = pivot_df["Date"].dt.year
pivot_df["Day"] = pivot_df["Date"].dt.strftime("%m-%d")  # You can choose a different date format if needed

In [ ]:
# Pivot the DataFrame
pivot_df = pivot_df.pivot(index="Day", columns="Year", values="VIX")

# Convert the year columns to strings to use them as dimension names
pivot_df.columns = pivot_df.columns.astype(str)

#Fill NA's (First with forward fill and then with backward fill)
# Apply fillna to all columns except "2023"
pivot_df[pivot_df.columns.difference(['2023'])] = pivot_df[pivot_df.columns.difference(['2023'])].fillna(method='ffill')
pivot_df = pivot_df.fillna(method='bfill')

#Reset Index
pivot_df = pivot_df.reset_index()

In [ ]:
#Add the Average to the Dataframe
pivot_df['Average'] = pivot_df.mean(numeric_only=True, axis=1)

# Create the Graph with VIX Average over the Year vs. Current Year

In [ ]:
#Limit the DataFrame to just 2023 and the Average
graph_df = pivot_df.drop(columns = {'Day'})

#Get a New columns for the day of the year
graph_df = graph_df.reset_index().rename(columns = {'index': 'Day'})
graph_df = graph_df.set_index('Day')

In [ ]:
# Generate all curves
def getCurves(n):
    for column in graph_df.columns: 
        color = 'black' if column == 'Average' else ('red' if column == '2023' else 'lightgrey')
        curve = hv.Curve(graph_df, 'Day', column).opts(
            opts.Curve(tools=['hover'], color=color, 
                       ylabel='VIX', width=700, show_legend=False))
        curve = curve.opts(xticks=10)
        yield column, curve  # Yield both the column name and the curve

source_curves, target_curves  = [], []
for col, curve in getCurves(2):
    tgt = curve.opts(width=700, ylabel='VIX')
    if col in ['2023', 'Average']:
        tgt = tgt.relabel(col)  # Set legend label using .relabel()
        tgt = tgt.opts(show_legend=True)  # Show legend only for desired columns
    target_curves.append(tgt)
    
overlaid_plot_tgt = hv.Overlay(target_curves)

overlaid_plot_tgt = overlaid_plot_tgt.relabel('VIX Index: Current Year vs. Average (1990-2023)').opts(
    height=400, legend_position='top')

# Layout the plot
full_graph = overlaid_plot_tgt

In [ ]:
#Save the Plot
p = pn.panel(full_graph)
p.save('Volatility_Seasonality_Curve_graph.html', embed = True)

# Manipulate the Data for the VIX Heatmap

In [ ]:
#Set column 'Date' as Index 
heatmap_df = df.set_index('Date')

#Resample the dataframe to Months and use the mean value for each month
heatmap_df = heatmap_df.resample('M').mean()

In [ ]:
#Reset Index to get the following formulas
heatmap_df = heatmap_df.reset_index()

#Create New columns with month and year for each row
heatmap_df['Year'] = pd.DatetimeIndex(heatmap_df['Date']).year
heatmap_df['Month'] = pd.DatetimeIndex(heatmap_df['Date']).month

#Convert Month from Integer to 3-letter name
heatmap_df['Month'] = pd.to_datetime(heatmap_df['Month'], format='%m').dt.month_name().str.slice(stop=3)

In [ ]:
#Convert table to Matrix Format
heatmap_df = heatmap_df.groupby(['Year','Month']).mean().unstack()
heatmap_df.columns = heatmap_df.columns.get_level_values(1)
heatmap_df = heatmap_df[['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']]

#Sort table by years ascending
heatmap_df = heatmap_df.sort_index(ascending=False)

#Round values to 2 decimal places
heatmap_df = heatmap_df.round(2)

In [ ]:
# Create a New Table with Statistics: Average Max, Mean, %Times Positive, %Times Negative
heatmap_stats_df = heatmap_df[0:0]
heatmap_stats_df.loc['Min.'] = heatmap_df.min()
heatmap_stats_df.loc['Avg.'] = heatmap_df.mean()
heatmap_stats_df.loc['Max.'] = heatmap_df.max()

heatmap_stats_df = heatmap_stats_df.round(2)

In [ ]:
#Convert the table to a style where Heatmap works

#For the Regular Data Series
heatmap_df_hv = heatmap_df.reset_index()
heatmap_df_hv = pd.melt(heatmap_df_hv, id_vars='Year', var_name='Month', value_name='VIX')
heatmap_df_hv = heatmap_df_hv[['Month','Year','VIX']]

#For the Statistics
heatmap_stats_df_hv = heatmap_stats_df.reset_index()
heatmap_stats_df_hv = pd.melt(heatmap_stats_df_hv, id_vars='Year', var_name='Month', value_name='VIX')
heatmap_stats_df_hv = heatmap_stats_df_hv[['Month','Year','VIX']]

# Creating the Heatmaps

## Create the Heatmap with the Regular Data

In [ ]:
#Define the Grids within the Heatmap
grid_style = {'grid_line_color': 'black', 'grid_line_width': 100}

#Define the Frame around the Heatmap
def hook(plot, element):
    plot.state.outline_line_width = 2
    plot.state.outline_line_color="black"
    
#Instantiate the heatmap
heatmap_vix = hv.HeatMap(heatmap_df_hv, label="VIX Index: Average Values by Month and Year")

heatmap_vix = heatmap_vix.opts(
    opts.HeatMap(width=700, height=800, xrotation=45, xaxis='top', labelled=[],
                 tools=['hover'], cmap='RdYlGn_r', 
                 yticks = heatmap_df_hv['Year'].to_list(), 
                 fontsize={'title': 15, 'xticks': 10, 'yticks': 10},
                 ))

heatmap_vix = heatmap_vix.opts(gridstyle=grid_style, show_grid=True, hooks=[hook])
heatmap_vix = heatmap_vix * hv.Labels(heatmap_vix).opts(padding=0)

In [ ]:
#Save the Plot
p = pn.panel(heatmap_vix)
p.save('Volatility_Seasonality_Regular_heatmap.html', embed = True)

## Create the Heatmap with the Statistics

In [ ]:
#Define the Grids within the Heatmap
grid_style = {'grid_line_color': 'black', 'grid_line_width': 100}

#Define the Frame around the Heatmap
def hook(plot, element):
    plot.state.outline_line_width = 2
    plot.state.outline_line_color="black"

heatmap_rows_list = ['Min.', 'Max.', 'Avg.']

heatmaps = []

for heatmap_row in heatmap_rows_list:
    data = heatmap_stats_df_hv[heatmap_stats_df_hv['Year'] == heatmap_row]
    heatmap = hv.HeatMap(data, label=f"Year {heatmap_row}")
    heatmap = heatmap.opts(
        opts.HeatMap(width=700, height=115, xrotation=45, xaxis='top', labelled=[],
                     tools=['hover'], cmap='RdYlGn_r',
                     fontsize={'title': 15, 'xticks': 10, 'yticks': 10},
                     ))
    heatmap = heatmap.opts(gridstyle=grid_style, show_grid=True, hooks=[hook])
    heatmap = heatmap * hv.Labels(heatmap).opts(padding=0)
    
    heatmaps.append(heatmap)
    
overlayed_heatmap = hv.Overlay(heatmaps, label="VIX 500 Index: Monthly Seasonality Statistics (1990-2023)").opts(opts.Overlay(show_legend=False, height=300, ))

In [ ]:
#Save the Plot
p = pn.panel(overlayed_heatmap)
p.save('Volatility_Seasonality_Statistics_heatmap.html', embed = True)